In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import os

In [12]:
path_fs = os.path.join("tmp_data", "db_fs.csv")
path_fca = os.path.join("tmp_data", "db_fca.csv")
path_faue = os.path.join("tmp_data", "db_faue.csv")

# db_fs find march churn uids

In [13]:
df_fs = pd.read_csv(path_fs)
df_fs.head(3)

,record_id,event_created_at_utc,start_date,end_date,sub_plan_id,sub_status_id,payment_number,sub_type_id,paid_period_days,merchant_id,...,usd_revenue,profit,cancelled_at,refund_price,ab_test_id,ab_group_id,app_id,platform_id,user_had_trial,autorenew_off_at
0,1999146,2021-12-08 15:48:02,2021-12-08 11:00:00,2021-12-15 11:00:00,30,3,0,3,7,1,...,0,0,NaN,0,28,6,3.0,2,True,2021-12-15 06:00:00
1,1999196,2021-12-09 04:00:09,2021-12-09 00:00:00,2022-01-09 00:00:00,33,2,21,3,31,1,...,799,679,NaN,0,28,6,2.0,2,True,NaN
2,2000623,2021-12-13 02:40:26,2021-12-12 22:00:00,2022-01-12 22:00:00,33,2,5,3,31,1,...,799,559,NaN,0,28,6,2.0,2,False,NaN


In [14]:
g_uid_fs = df_fs.groupby("user_id")

buff = (g_uid_fs.end_date.last() < "2022-01-01 23:59:59") & (g_uid_fs.start_date.last() > "2021-12-01 00:00:00")
buff


user_id
63         False
71         False
136        False
190        False
208        False
           ...  
2260411    False
2260417    False
2260430    False
2260454    False
2260483    False
Length: 112658, dtype: bool

In [15]:
dec_active_jan_churn = buff.index[buff == True]
dec_active_jan_churn


Int64Index([ 122044,  755215, 1008731, 1098977, 1225931, 1265602, 1322489,
            1450346, 1459667, 1522815,
            ...
            2230067, 2230081, 2230106, 2230111, 2230170, 2230563, 2230720,
            2231422, 2232083, 2232832],
           dtype='int64', name='user_id', length=620)

In [16]:
churned_in_jan = pd.Series(dec_active_jan_churn)
churned_in_jan.to_csv('churned_in_jan.csv', header= False, index= False)

# -----------------------------------------------------------------

In [8]:
buff = (g_uid_fs.end_date.last() > "2022-01-01 23:59:59")
buff

user_id
63         False
71         False
136        False
190        False
208        False
           ...  
2260411     True
2260417     True
2260430     True
2260454     True
2260483     True
Name: end_date, Length: 112658, dtype: bool

In [9]:
usrs_need_to_drop = buff.index[buff == True]
usrs_need_to_drop

Int64Index([    439,    4300,    4534,    8207,   39533,   45211,   53235,
              54376,   61380,   77230,
            ...
            2260297, 2260311, 2260317, 2260364, 2260392, 2260411, 2260417,
            2260430, 2260454, 2260483],
           dtype='int64', name='user_id', length=5899)

In [10]:
buff = pd.Series(usrs_need_to_drop)
buff.to_csv("usrs_need_to_drop.csv", header= False, index= False)

# -------------------------------------------------------------------------------------------

## Drop some users from global table

In [46]:
df_gl = pd.read_csv('global_usrs.csv', names = ['user_id', 'periods_count', 'first_plan_id', 'last_plan_id', 'bool_changed_plan', 'first_app_id', 'last_app_id', 'first_platform_id', 'last_platform_id', 'last_subs_status', 'first_merchant', 'last_merchant', 'bool_changed_merchant', 'used_coupons_count', 'user_had_trial', 'total_profit', 'bool_autorenew_off', 'count_course_activity_id', 'median_cource_grade_id', 'count_content_id', 'kids_amount', 'total_usage_steps', 'bool_isMember_of_campaign', 'bool_is_school'], header= None)
df_gl

,user_id,periods_count,first_plan_id,last_plan_id,bool_changed_plan,first_app_id,last_app_id,first_platform_id,last_platform_id,last_subs_status,...,user_had_trial,total_profit,bool_autorenew_off,count_course_activity_id,median_cource_grade_id,count_content_id,kids_amount,total_usage_steps,bool_isMember_of_campaign,bool_is_school
0,63,2,58,58,False,1.0,1.0,2,2,1,...,True,1399,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,71,24,58,58,False,1.0,1.0,2,2,1,...,True,35477,0,261.0,2.0,261.0,1.0,86.0,0.0,1.0
2,136,9,58,58,False,1.0,1.0,2,2,1,...,True,11192,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,190,1,58,58,False,1.0,1.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,208,1,43,43,False,1.0,1.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106754,2235282,2,36,36,False,2.0,2.0,2,2,1,...,True,5599,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
106755,2235692,1,36,36,False,2.0,2.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0
106756,2236075,1,30,30,False,3.0,3.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0
106757,2238533,1,32,32,False,3.0,3.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [47]:
df_gl = df_gl.drop(df_gl[np.isin( df_gl.user_id.values, usrs_need_to_drop)].index)
df_gl

,user_id,periods_count,first_plan_id,last_plan_id,bool_changed_plan,first_app_id,last_app_id,first_platform_id,last_platform_id,last_subs_status,...,user_had_trial,total_profit,bool_autorenew_off,count_course_activity_id,median_cource_grade_id,count_content_id,kids_amount,total_usage_steps,bool_isMember_of_campaign,bool_is_school
0,63,2,58,58,False,1.0,1.0,2,2,1,...,True,1399,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,71,24,58,58,False,1.0,1.0,2,2,1,...,True,35477,0,261.0,2.0,261.0,1.0,86.0,0.0,1.0
2,136,9,58,58,False,1.0,1.0,2,2,1,...,True,11192,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,190,1,58,58,False,1.0,1.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,208,1,43,43,False,1.0,1.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106754,2235282,2,36,36,False,2.0,2.0,2,2,1,...,True,5599,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
106755,2235692,1,36,36,False,2.0,2.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0
106756,2236075,1,30,30,False,3.0,3.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,0.0,1.0,0.0,1.0
106757,2238533,1,32,32,False,3.0,3.0,2,2,3,...,True,0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


# Make predictions

In [65]:
from model_layer import Models_ensemble

ens = Models_ensemble()
await ens.make_predictions(df_gl)
ens.preds

c:\DATA\Projects\Model_testing_for_Kids\.venv\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\DATA\Projects\Model_testing_for_Kids\.venv\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.23.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\DATA\Projects\Model_testing_for_Kids\.venv\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.2

,user_id,LogisticRegression(random_state=14),RandomForestClassifier(random_state=14),GradientBoostingClassifier()
0,63,True,True,True
1,71,True,True,True
2,136,True,True,True
3,190,True,True,True
4,208,True,True,True
...,...,...,...,...
106754,2235282,False,False,False
106755,2235692,True,True,True
106756,2236075,True,True,True
106757,2238533,True,False,True


In [66]:
ens.preds.to_csv('res_1.csv', index= False)

# -------------------------------------------------------------------------------------------------

# Accuracy 

In [4]:
res = pd.read_csv('res_1.csv')
res.columns

Index(['user_id', 'LogisticRegression(random_state=14)',
       'RandomForestClassifier(random_state=14)',
       'GradientBoostingClassifier()'],
      dtype='object')

## Logit acc

In [22]:
logit_c_uid = res[(res['LogisticRegression(random_state=14)'] == True)].user_id

np.isin(churned_in_jan, logit_c_uid).sum() / churned_in_jan.shape[0]

0.8919354838709678

## Rforest acc

In [24]:
rforest_c_uid = res[(res['RandomForestClassifier(random_state=14)'] == True)].user_id

np.isin(churned_in_jan, rforest_c_uid).sum() / churned_in_jan.shape[0]

0.8935483870967742

## GBtree acc

In [25]:
gbtree_c_uid = res[(res['GradientBoostingClassifier()'] == True)].user_id

np.isin(churned_in_jan, gbtree_c_uid).sum() / churned_in_jan.shape[0]

0.9